1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [1]:
!pip install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.56.0
    Uninstalling google-cloud-aiplatform-1.56.0:
      Successfully uninstalled google-cloud-aiplatform-1.56.0


2. Use the following code in your application to request a model response

In [1]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def generate():
  vertexai.init(project="peaceful-harbor-393817", location="us-east1")
  model = GenerativeModel(
    "gemini-1.5-flash-001",
  )
  responses = model.generate_content(
      ["""Multi-choice problem: Define the category of the ticket?
Categories:
- Credit card
- Bank account services
- Loans and Mortgages

Please only print the category name without anything else.

Ticket: I lost my credit card numbered 12345. Can you help with deactivating the card?
Category: Credit card

Ticket: I would like to change the address associated with my account. I have been calling the bank multiple times but couldn\'t get through. Please help me.
Category: Bank account services

Ticket: good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase bank cardmember services. I wrote to chase asking for debt verification and what they sent me a statement which is not acceptable i am asking the bank to validate the debt instead i been receiving mail every month from them attempting to collect a debt i have a right to know this information as a consumer chase account xxxx xxxx xxxx xxxx thanks in advance for your help
Category: Loans and Mortgages

Ticket: my grand son give me check for i deposit it into my chase account after fund clear my chase bank closed my account never paid me my money they said they need to speak with my grand son check was clear money was taking by my chase bank refuse to pay me my money my grand son called chase times they told him i should call not him to verify the check owner he is out the country most the time date happen check number xxxx claim number is xxxx with chase
Category:
"""],
      generation_config=generation_config,
      safety_settings=safety_settings,
  )

  print(responses)


generation_config = {
    "max_output_tokens": 256,
    "temperature": 0.2,
    "top_p": 0.8,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()



candidates {
  content {
    role: "model"
    parts {
      text: "Bank account services \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.17981843650341034
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.09999629855155945
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: LOW
    probability_score: 0.6363531351089478
    severity: HARM_SEVERITY_LOW
    severity_score: 0.23792988061904907
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.1139734759926796
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.12421301752328873
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.1139734759926796
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.029986508190631866
  }
}
usage_metadata {
  prompt

In [2]:
from google.colab import files
uploaded = files.upload()


Saving bankingdataset.csv to bankingdataset.csv


In [3]:
import pandas as pd
encoding_list = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

for encoding in encoding_list:
    try:
        banking_df = pd.read_csv("/content/bankingdataset.csv", encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        print(banking_df.head())
        break
    except UnicodeDecodeError as e:
        print(f"Failed to read the file with encoding: {encoding}. Error: {e}")


Failed to read the file with encoding: utf-8. Error: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Successfully read the file with encoding: latin1
   ÿ"customer_id" call_date call_time  \
0      1234567890  31-10-23  13:55:00   
1      3456789012  31-10-23  14:05:00   
2      1098765432  31-10-23  14:10:00   
3      2134567890  31-10-23  14:15:00   
4      4567890123  31-10-23  14:20:00   

                                     call_transcript  
0  Customer: Hello, I'm calling to check on the s...  
1  Customer: Hello, I'm calling to inquire about ...  
2  Customer: Hello, I'm calling to transfer money...  
3  Customer: Hello, I'm calling to dispute a char...  
4  Customer: Hello, I'm calling to ask about your...  


In [ ]:
import pandas as pd
import time

# Initialize your Vertex AI model
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project="peaceful-harbor-393817", location="us-east1")
model = GenerativeModel("gemini-1.5-flash-001")

# Function to classify the category of a ticket
def classify_ticket(call_transcript):
    prompt = f"""Multi-choice problem: Define the category of the ticket?
Categories:
- Credit card
- Bank account services
- Loans and Mortgages+

Ticket: {call_transcript}
Category: """
    response = model.generate_content(prompt)
    return response.text.strip()  # Assuming response has a text attribute


# Apply the classification function to each row with rate limiting
classifications = []
for index, row in banking_df.iterrows():
    try:
        classification = classify_ticket(row['call_transcript'])
        classifications.append(classification)
        # print(f"Processed row {index}: {classification}")
        time.sleep(1)  # Sleep for 1 second to avoid exceeding the quota
    except Exception as e:
        # print(f"Error processing row {index}: {e}")
        classifications.append(None)  # Append None if there's an error

banking_df["Classification"] = classifications

print(banking_df.head())
